<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good at cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [2]:
# Import libraries here.
import numpy as np
import pandas as pd
import os

The `os` library ([https://docs.python.org/3/library/os.html](https://docs.python.org/3/library/os.html)) is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [8]:
# Create an output folder.

try:
    os.mkdir('./output')
except:
    print(Output folder already exists! Moving on...')

# Create a files variable that contains all of our data files.

files = os.listdir('./data') 

FileExistsError: Output folder already exists! Moving on...


### Step 2: Process one data frame (one sheet from one file)
It looks like we have data for the month of January--31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.) to take a look at what you'll be reading in.

In [3]:
# Read in data from only ONE city (your city or one city of choice) from January 1st.

chicago = pd.read_excel('./data/Jan 1.xlsx', sheet_name='Chicago')

In [4]:
chicago.columns

Index(['prodcode', 'price_eu', 'weight_kg', 'quantity'], dtype='object')

### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [5]:
eu_to_usd = lambda x: x * 1.1
chicago['price_usd'] = chicago['price_eu'].apply(eu_to_usd)

In [6]:
kg_to_lb = lambda x: x * 2.2
chicago['weight_lb'] = chicago['weight_kg'].apply(kg_to_lb)

### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data from above.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [7]:
plu = pd.read_csv("plu-codes.csv")
plu.rename(columns={'plu_code': 'prodcode'}, inplace=True)

In [8]:
chicago = pd.merge(chicago, plu, how='inner', on='prodcode')

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [9]:
chicago.drop(columns=['price_eu', 'weight_kg'], inplace=True)

### Step 2d: Add the date
Simply create a new `date` column for this dataframe that contains the date this data was collected. For example, this data is from `Jan 1.xlsx`, so this column should be full of `Jan 1`.  
_Note: In this step, this does not need to be an automated function that would assign the relevant date for data from any file. Simply create this one column._

In [10]:
chicago['date'] = 'Jan 1'

### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [11]:
def process_data(file, city):
    # Read in data
    city = pd.read_excel(f'./data/{file}', sheet_name=city)
    
    # Conversions for USD, lb
    city['price_usd'] = city['price_eu'].apply(eu_to_usd)
    city['weight_lb'] = city['weight_kg'].apply(kg_to_lb)
    
    # Merge in product names
    city = pd.merge(city, plu, how='inner', on='prodcode')
    
    # Drop unneccessary columns
    city.drop(columns=['price_eu', 'weight_kg'], inplace=True)
    
    # Add date column
    city['date'] = file.split('.')[0]
    
    # Return processed dataframe
    return city

Test your function out on a new file and city (one sheet from that file)!

In [12]:
boston = process_data('Jan 1.xlsx', 'Boston')

### Step 4: Process all of January's data for your city
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [13]:
chicago = [process_data(filename, 'Chicago') for filename in files]

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [14]:
chicago = pd.concat(chicago).reset_index(drop=True)

In [15]:
chicago

prodcode  quantity  price_usd  weight_lb   product   date
0         4081       334   1.921072   7.568684  Eggplant  Jan 1
1         4081       477   3.067273   3.319229  Eggplant  Jan 1
2         4081       118   1.838032  16.957144  Eggplant  Jan 1
3         4081       278   3.084451  16.243687  Eggplant  Jan 1
4         4081       333   1.762335  10.818943  Eggplant  Jan 1
...        ...       ...        ...        ...       ...    ...
4657      4030       415   1.852966  16.329549      Kiwi  Jan 9
4658      4030       130   0.630705  12.810045      Kiwi  Jan 9
4659      4958       461   0.609706   5.493598     Lemon  Jan 9
4660      4958       344   0.851364   8.333432     Lemon  Jan 9
4661      4958       360   0.919408   6.089030     Lemon  Jan 9

[4662 rows x 6 columns]

### Step 6: Do this for all cities, write data to your output folder
Here's the big one. For each city, process the data as in steps 3-5, and then write the data to your `output` folder. Below is a dictionary of city names to desired output file names.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [16]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [17]:
%%time
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

for city, csv in city_dict.items():
    city_df_list = [process_data(filename, city) for filename in files] # 
    city_df = pd.concat(city_df_list).reset_index(drop=True) # concatenate list of dataframes
    city_df['city'] = city # New column with city name
    city_df = city_df.loc[:, ["city", "date", "product", "prodcode", "quantity", "weight_lb", "price_usd"]]
    city_df.to_csv(f'./output/{csv}')

Wall time: 22.7 s


# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list _(Hint: Using the `os` library and a list comprehension would make this 1-2 lines of code!)_
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [19]:
all_cities = [pd.read_csv(f'./output/{csv_name}') for csv_name in os.listdir('./output') if '.csv' in csv_name]

In [20]:
all_cities = pd.concat(all_cities).reset_index(drop=True)

In [23]:
all_cities.groupby('city').mean()[['quantity', 'weight_lb', 'price_usd']]

quantity  weight_lb  price_usd
city                                            
Atlanta         302.485306  11.929027   1.937980
Austin          301.048298  12.092775   1.937456
Boston          298.806536  12.063057   1.900973
Chicago         301.686186  12.101555   1.930026
Denver          301.012745  12.129850   1.932088
Los Angeles     301.531276  12.167527   1.918331
New York        299.482863  12.090294   1.912662
San Francisco   298.979895  12.237399   1.925830
Seattle         300.333694  11.925486   1.903315
Washington, DC  300.576421  11.930747   1.943107

# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file (editable in any text editor) and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - **Comment your code.** Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).